# Predicting Stock Prices
The goal of this project is to train a model capable of predicting future closing prices for the S&P500 index using a daily record of index prices from 1950 to 2015.

Since the goal is to predict future events, care must be taken to avoid features which 'leak' data from the future. This would train the model with the same data that it is being trained to predict.

## Data read-in

In [1]:
import pandas as pd
data = pd.read_csv("sphist.csv")
data_bkp = data.copy()

## Data overview

In [2]:
data.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
Date         16590 non-null object
Open         16590 non-null float64
High         16590 non-null float64
Low          16590 non-null float64
Close        16590 non-null float64
Volume       16590 non-null float64
Adj Close    16590 non-null float64
dtypes: float64(6), object(1)
memory usage: 907.3+ KB


In [4]:
data.describe()

,Open,High,Low,Close,Volume,Adj Close
count,16590.000000,16590.000000,16590.000000,16590.000000,1.659000e+04,16590.000000
mean,482.570941,485.624237,479.367501,482.692491,7.940099e+08,482.692491
std,554.889186,558.186049,551.367625,555.007904,1.456582e+09,555.007904
min,16.660000,16.660000,16.660000,16.660000,6.800000e+05,16.660000
25%,83.860001,84.594997,83.139997,83.860001,7.610000e+06,83.860001
50%,144.049995,145.294998,143.105004,144.264999,7.170500e+07,144.264999
75%,950.722488,956.665024,941.969986,950.797500,7.866750e+08,950.797500
max,2130.360107,2134.719971,2126.060059,2130.820068,1.145623e+10,2130.820068


## Feature engineering
As described in the introduction, the intended function of the model is a predictive one. Any future knowledge must be removed from the training data to avoid a model that is deceptively good when trained and tested, but that'll fail when applied to real data.

To do so two methods are apparent:
- Shift the data so that each closing price entry is paired with the price indices of the previous day.
- Create new features that describe historical data in some way and remove the existing ones.

Both approaches will be described in the following.

Additionally, the Date column is given as a string. Converting it into a datetime object will allow for date comparisons and ordering. Furthermore, dates could be converted into an ordinal, epoch-based format, and used directly as inputs for the model. Finally the day of the week could be extracted and used as a categorical feature.

In [5]:
from datetime import datetime, timedelta
# Convert dates to datetime
data["Date"] = data["Date"].apply(pd.to_datetime)
# Generate weekday columns
data["Weekday"] = data["Date"].apply(lambda x: x.weekday())
data = pd.concat([data,pd.get_dummies(data["Weekday"], prefix="Weekday", prefix_sep=" ")],axis=1)
del data["Weekday"]
# Convert dates to ordinal
data["Ord Date"] = data["Date"].apply(lambda x: x.toordinal())
# Sort from least to most recent
data = data.sort_values("Date", ascending = True)
data.head()

,Date,Open,High,Low,Close,Volume,Adj Close,Weekday 0,Weekday 1,Weekday 2,Weekday 3,Weekday 4,Ord Date
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,0,1,0,0,0,711860
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,0,0,1,0,0,711861
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,0,0,0,1,0,711862
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,0,0,0,0,1,711863
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,1,0,0,0,0,711866


### Data shift - Approach A
The first approach to be tested will be the data shift (Approach A). A new column will be added by copying the Close column and shifting it up by one day: the new column will be the new target to be predicted. One datapoint will be lost in the process.

In [6]:
adata = data.copy()
adata["Shft Close"] = adata["Close"].shift(periods=-1)
adata = adata.dropna(axis=0)
adata.head()

,Date,Open,High,Low,Close,Volume,Adj Close,Weekday 0,Weekday 1,Weekday 2,Weekday 3,Weekday 4,Ord Date,Shft Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,0,1,0,0,0,711860,16.850000
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,0,0,1,0,0,711861,16.930000
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,0,0,0,1,0,711862,16.980000
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,0,0,0,0,1,711863,17.080000
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,1,0,0,0,0,711866,17.030001


### Feature creation - Approach B
The second approach to be tested will be the creation of new features from historical data (Approach B). Very many features could created - the following will focus on three of them:
- Average price in the previous 5 days of trading
- Average price in the last year
- Ratio of the two averages

In doing so one year of data will be lost since the new features need one entire year of previous data to be created.

In [7]:
bdata = data.copy()
for i in range(data.shape[0]):
    # Create column for the average price in the previous 5 days of trading
    bdata.at[bdata.index[i],"Avg 5d"] = bdata.iloc[i-5:i,4].mean()
    # Create column for the average price in the last year
    if bdata.iat[i,0] >= bdata.iat[0,0]+timedelta(days=365): # If datapoint beyond first 365 days, then average previous 365 days
        bdata.at[bdata.index[i],"Avg 1y"] = bdata.loc[bdata["Date"].between(bdata.iat[i,0]-timedelta(days=365),bdata.iat[i,0]-timedelta(days=1)), "Close"].mean()
    else: # Else output NaN
        bdata.at[bdata.index[i],"Avg 1y"] = float('nan')

In [8]:
# Drop rows with NaN values
bdata = bdata.dropna(axis=0)
# Create column for the ratio of the averages
bdata["Avg Ratio"] = bdata["Avg 5d"]/bdata["Avg 1y"]

In [9]:
bdata.head()

,Date,Open,High,Low,Close,Volume,Adj Close,Weekday 0,Weekday 1,Weekday 2,Weekday 3,Weekday 4,Ord Date,Avg 5d,Avg 1y,Avg Ratio
16339,1951-01-03,20.690001,20.690001,20.690001,20.690001,3370000.0,20.690001,0,0,1,0,0,712225,20.360000,18.406760,1.106115
16338,1951-01-04,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,0,0,0,1,0,712226,20.514000,18.422880,1.113507
16337,1951-01-05,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,0,0,0,0,1,712227,20.628000,18.438960,1.118718
16336,1951-01-08,21.000000,21.000000,21.000000,21.000000,2780000.0,21.000000,1,0,0,0,0,712230,20.726001,18.460643,1.122713
16335,1951-01-09,21.120001,21.120001,21.120001,21.120001,3800000.0,21.120001,0,1,0,0,0,712231,20.840001,18.470800,1.128267


## Model implementation
The next step is the model implementation proper. The model will be applied to Dataset A first, then to Dataset B.

To simulate the actual application of the model, the implementation will be as follows:
- A training/testing threshold date will be chosen.
- All data previous to the threshold date will be used to train the model.
- The trained model will be used to predict the closing price on the threshold date.
- The predicted price will be stored, the training set will then be shifted up one day to include the actual closing price for the threshold date.
- The threshold date will be moved up one day and the entire process will be repeated.

This simulates the behaviour of a user employing the model daily to predict next-day trade. At the end of the cycle predicted and actual prices will be compared to produce a final error metric for the model.

In [10]:
# Import the model
from sklearn.linear_model import LinearRegression

In [11]:
# A useful function for rescaling numeric columns as a DataFrame
def df_minmax_scale(df):
    return (df-df.min())/(df.max()-df.min())
# A useful function for splitting the data into training and test sets following a user-given date threshold
def train_test_split(threshold, features, target, df):
    temp = df.copy()
    temp[features] = df_minmax_scale(temp[features])
    xtrain = temp.loc[temp["Date"] < threshold, features]
    ytrain = temp.loc[temp["Date"] < threshold, target]
    xtest = temp.loc[temp["Date"] == threshold, features]
    ytest = temp.loc[temp["Date"] == threshold, target]
    return xtrain, ytrain, xtest, ytest

### Approach A

In [12]:
# Select the days over which to cycle
days = adata.loc[adata["Date"] >= datetime(year=2013,month=1,day=1), "Date"]
# Select the features
features = ["Open","High","Low","Close","Volume","Weekday 0","Weekday 1","Weekday 2","Weekday 3","Weekday 4","Ord Date"]
# Select the target
target = "Shft Close"
# Prepare Series objects that'll contain actual and predicted values
actual = []
predicted = []
# Cycle over all selected days and predict a next-day closing price for each
for d in days:
    xtrain, ytrain, xtest, ytest = train_test_split(d, features, target, adata)
    lr = LinearRegression()
    lr.fit(xtrain,ytrain)
    prediction = lr.predict(xtest)
    actual.append(ytest.iat[0]) # Extract the value from the single-element Series
    predicted.append(prediction[0]) # Extract the value from the single-element array

The final step is to choose an error metric to calculate. Common choices are the Root Mean Squared Error (RMSE) and the Mean Absolute Error (MAE).

In [13]:
from numpy import sqrt, mean, array
# Convert to array to streamline calculation
a = array(actual)
b = array(predicted)
# Calculate RMSE and MAE
RMSE = sqrt(mean(((a-b)**2)))
MAE = mean(abs(a-b))
print("RMSE: ", RMSE, "\nMAE: ", MAE)

RMSE:  15.125919620088471 
MAE:  11.015680084673813


### Approach B

In [14]:
# Select the days over which to cycle
days = bdata.loc[bdata["Date"] >= datetime(year=2013,month=1,day=1), "Date"]
# Select the features
features = ["Weekday 0","Weekday 1","Weekday 2","Weekday 3","Weekday 4","Avg 5d","Avg 1y","Avg Ratio","Ord Date"]
# Select the target
target = "Close"
# Prepare Series objects that'll contain actual and predicted values
actual = []
predicted = []
# Cycle over all selected days and predict a next-day closing price for each
for d in days:
    xtrain, ytrain, xtest, ytest = train_test_split(d, features, target, bdata)
    lr = LinearRegression()
    lr.fit(xtrain,ytrain)
    prediction = lr.predict(xtest)
    actual.append(ytest.iat[0]) # Extract the value from the single-element Series
    predicted.append(prediction[0]) # Extract the value from the single-element array

Once again the final step is to calculate the RMSE and MAE.

In [15]:
from numpy import sqrt, mean, array
# Convert to array to streamline calculation
a = array(actual)
b = array(predicted)
# Calculate RMSE and MAE
RMSE = sqrt(mean(((a-b)**2)))
MAE = mean(abs(a-b))
print("RMSE: ", RMSE, "\nMAE: ", MAE)

RMSE:  22.206226206921052 
MAE:  16.153862199111177


## Results
Approach A was shown to produce the best results, with RMSE and MAE values equal to 15.126 and 11.017 respectively. However, 'Avg 5d', 'Avg 1y' and 'Avg Ratio' are only few of many features which could be constructed from the available data. New and better features may ultimately improve the accuracy of Approach B.

# Automatic data retrieval and prediction for tomorrow
With a functioning model capable of predicting closing prices, the logical next step is to get actual data and predict the closing price for tomorrow. [Yahoo Finance](https://finance.yahoo.com) offers historical stock market data as free downloads.

Unfortunately, starting on May 2017 Yahoo Finance terminated its free EOD data download service, leaving many of the previous data recovery libraries non-functional. The new data download system works around a cookie and a "crumb" for authentication (reference [here](https://github.com/c0redumb/yahoo_quote_download/blob/master/README.md)): the [yqd](https://github.com/c0redumb/yahoo_quote_download/tree/master/yahoo_quote_download) library ([credit: @c0redumb](https://github.com/c0redumb)) tackles this by querying Yahoo Finance and extracting the "crumb" from the response - the cookie is then extracted from the cookiejar and both are used to retrieve the data through a bespoke function.

In [16]:
ticker = "^GSPC" # The symbol for the S&P 500 index on Yahoo Finance
# Start and end dates are given as strings with format "YYYY-MM-DD"
start = "19700101"
end = "{0}{1}{2}".format(datetime.now().year,datetime.now().month,datetime.now().day)
import yqd
nu_data = yqd.load_yahoo_quote(ticker,start,end,format_output='dataframe')

In [17]:
nu_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1970-01-02,92.059998,93.540001,91.790001,93.000000,93.000000,8050000
1,1970-01-05,93.000000,94.250000,92.529999,93.459999,93.459999,11490000
2,1970-01-06,93.459999,93.809998,92.129997,92.820000,92.820000,11460000
3,1970-01-07,92.820000,93.379997,91.930000,92.629997,92.629997,10010000
4,1970-01-08,92.629997,93.470001,91.989998,92.680000,92.680000,10670000


In [18]:
nu_data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
12290,2018-09-19,2906.600098,2912.360107,2903.820068,2907.949951,2907.949951,3280020000
12291,2018-09-20,2919.729980,2934.800049,2919.729980,2930.750000,2930.750000,3337730000
12292,2018-09-21,2936.760010,2940.909912,2927.110107,2929.669922,2929.669922,5607610000
12293,2018-09-24,2921.830078,2923.790039,2912.629883,2919.370117,2919.370117,3372210000
12294,2018-09-25,2921.750000,2923.949951,2919.000000,2920.989990,2920.989990,492728715


With the new data available, the training and prediction process can be simply repeated. This time however the entire dataset minus the last line is used for prediction, and the prediction is cast on the last line only. 

In [19]:
# Convert dates to datetime
nu_data["Date"] = nu_data["Date"].apply(pd.to_datetime)
# Generate weekday columns
nu_data["Weekday"] = nu_data["Date"].apply(lambda x: x.weekday())
nu_data = pd.concat([nu_data,pd.get_dummies(nu_data["Weekday"], prefix="Weekday", prefix_sep=" ")],axis=1)
del nu_data["Weekday"]
# Convert dates to ordinal
nu_data["Ord Date"] = nu_data["Date"].apply(lambda x: x.toordinal())
# Sort from least to most recent
nu_data = nu_data.sort_values("Date", ascending = True)
nu_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Weekday 0,Weekday 1,Weekday 2,Weekday 3,Weekday 4,Ord Date
0,1970-01-02,92.059998,93.540001,91.790001,93.000000,93.000000,8050000,0,0,0,0,1,719164
1,1970-01-05,93.000000,94.250000,92.529999,93.459999,93.459999,11490000,1,0,0,0,0,719167
2,1970-01-06,93.459999,93.809998,92.129997,92.820000,92.820000,11460000,0,1,0,0,0,719168
3,1970-01-07,92.820000,93.379997,91.930000,92.629997,92.629997,10010000,0,0,1,0,0,719169
4,1970-01-08,92.629997,93.470001,91.989998,92.680000,92.680000,10670000,0,0,0,1,0,719170


In [20]:
adata = nu_data.copy()
adata["Shft Close"] = adata["Close"].shift(periods=-1)
# Note that we're not dropping the line containing the NaN value since this is the one we want to predict on
adata.tail(1)

,Date,Open,High,Low,Close,Adj Close,Volume,Weekday 0,Weekday 1,Weekday 2,Weekday 3,Weekday 4,Ord Date,Shft Close
12294,2018-09-25,2921.750000,2923.949951,2919.000000,2920.989990,2920.989990,492728715,0,1,0,0,0,736962,NaN


In [21]:
# Select the features
features = ["Open","High","Low","Close","Volume","Weekday 0","Weekday 1","Weekday 2","Weekday 3","Weekday 4","Ord Date"]
# Select the target
target = "Shft Close"
# No cycle this time, just a single prediction
temp = adata.copy() # Get the data into a copy DataFrame
temp[features] = temp[features].astype(float) # Cast features as float type - the data download retrieves them as strings!
temp[features] = df_minmax_scale(temp[features]) # Rescale
# Assign training and testing sets
test = temp.tail(1)
train = temp.dropna(axis=0)
xtrain = train[features]
ytrain = train[target]
xtest = test[features]
# Predict
lr = LinearRegression()
lr.fit(xtrain,ytrain)
prediction = lr.predict(xtest)[0]
change = prediction - float(adata.tail(1)["Close"].iat[0])
print("Predicted S&P 500 closing price for tomorrow: ", prediction, "\nChange from today's closing price: ", change)

Predicted S&P 500 closing price for tomorrow:  2921.3056667515875 
Change from today's closing price:  0.315676751587489
